In [1]:
import numpy as np
import qutip as qt
from scipy.linalg import logm, expm
from qiskit.quantum_info import Operator, state_fidelity
from qiskit import QuantumCircuit
from copy import deepcopy

from tools.classical import *
from tools.quantum import *

#### Qutip Exact Hamiltonian

In [3]:
num_qubits = 3

X = qt.sigmax()
Y = qt.sigmay()
Z = qt.sigmaz()
coeffs = [1, 1, 1, 1]
hamiltonian_qt = hamiltonian_matrix([X, X], [Y, Y], [Z, Z], coeffs=coeffs, num_qubits=num_qubits, symbreak_term=[Z])
# print(hamiltonian_qt)
#* HAMILTONIANS
exact_spec = np.linalg.eigvalsh(hamiltonian_qt)
rescaling_factor, shift = rescaling_and_shift_factors(hamiltonian_qt)  # Shift already rescaled
rescaled_hamiltonian_qt = hamiltonian_qt / rescaling_factor + shift * qt.qeye(hamiltonian_qt.shape[0])

#* SPECTRUMS
# print(f'Shifted spectrum {np.linalg.eigvalsh(shifted_hamiltonian_qt)}')
print(f'Rescaling factor {rescaling_factor}')
print(f'Shift {shift}')
print(f'Original spectrum {exact_spec}')
print(f'Normalized spectrum {np.linalg.eigvalsh(rescaled_hamiltonian_qt)}')

Rescaling factor 8.0
Shift 0.5
Original spectrum [-4.         -3.46410162 -2.82842712 -2.          2.          2.82842712
  3.46410162  4.        ]
Normalized spectrum [-7.11644431e-17  6.69872981e-02  1.46446609e-01  2.50000000e-01
  7.50000000e-01  8.53553391e-01  9.33012702e-01  1.00000000e+00]


### Qutip unitary evolution construction and comparison: Exact vs Trotterized

$n = 3, k = 4$
Hamiltonian is rescaled already by its coupling strength, not by the rescaled time anymore (bit easier to have an overview)
$$H = 0.125\times Heisenberg_3 + Z_{bulk}$$

In [4]:
T = 1
total_time = T * 2 * np.pi
num_trotter_steps = 100
step_size = total_time / num_trotter_steps
rescaled_coeffs = coeffs / rescaling_factor

rescaled_total_time = total_time / rescaling_factor
rescaled_step_size = rescaled_total_time / num_trotter_steps

#* EXACT UNITARIES
exact_U_qt = expm(1j * rescaled_hamiltonian_qt * total_time)

#* QUTIP TROTTER
trott_U_qt = trotter_heisenberg_qutip(num_qubits, step_size, num_trotter_steps, coeffs=rescaled_coeffs, shift=(total_time*shift))
# UUdag_minus_id = exact_U_qt * trott_U_qt.dag() - qt.qeye(exact_U_qt.shape[0])

#* QUTIP DISTANCES
dist_between_unitaries = np.linalg.norm(exact_U_qt - trott_U_qt)
print(f'Distance between Qutip Trotter vs Exact {dist_between_unitaries}')

# with np.printoptions(precision=3, suppress=True):
#     print(exact_U_qt)
#     print(trott_U_qt)
#     print(UUdag_minus_id)
    
# logU_shifted = logm(shifted_trott_U_qt) / (1j * T)
# logU = logm(trott_U_qt) / (1j * T)
# dist_shifted = np.linalg.norm(logU_shifted - shifted_exact_U_qt)
# dist = np.linalg.norm(logU - exact_U_qt)
# print(dist_shifted)
# print(dist)

Distance between Qutip Trotter vs Exact 0.04150634298227714


### Observable expectation comparison: Exact vs Trotterized, finding a good number of Trotter steps

In [5]:
#* Initial state
# initial_state = np.ones(2**num_qubits) / np.sqrt(2**num_qubits)  # Equal superposition
np.random.seed(666)
randstate_better = np.zeros(2**num_qubits)
randstate_better[np.random.choice(2**num_qubits, 2**num_qubits//2, replace=False)] = 1
print(randstate_better)
randstate_better /= np.linalg.norm(randstate_better)
rand_ini_state = randstate_better
rand_ini_state_qt = rand_ini_state

initial_energy = rand_ini_state.conj().T @ rescaled_hamiltonian_qt.data @ rand_ini_state
print(f'Initial energy {initial_energy}')

# Pretty garbage:
# rand_state0 = np.random.random(2**num_qubits) + 1j * np.random.random(2**num_qubits)  # Random state
# rand_state0 /= np.linalg.norm(rand_state0)
# rand_state1 = np.random.random(2**num_qubits) + 1j * np.random.random(2**num_qubits)  # Random state
# rand_state1 /= np.linalg.norm(rand_state1)
# print(qt.fidelity(qt.Qobj(randstate_better), qt.Qobj(rand_state1)))

#* Evolved states
exact_evolved_state = qt.Qobj(exact_U_qt) * qt.Qobj(rand_ini_state_qt)
trott_evolved_state = qt.Qobj(trott_U_qt) * qt.Qobj(rand_ini_state_qt)

#* Fidelities
exact_evolved_fidelity = qt.fidelity(exact_evolved_state, trott_evolved_state)
# print('Shifted:')  #* Global phase won't show up in fidelity
# fid_of_states_shifted = qt.fidelity(exact_evolved_state_shifted, trott_evolved_state_shifted)
print(f'Exact-Evolved fidelity {exact_evolved_fidelity}')


[1. 0. 0. 1. 0. 0. 1. 1.]
Initial energy (0.6875+0j)
Exact-Evolved fidelity 0.9999394380316122


#### Circuit

In [23]:
shift_1q = lambda shift: np.diag(np.ones(2) * np.exp(1j * shift))

In [24]:
trotter_step = trotter_step_heisenberg(num_qubits, coeffs=coeffs, symbreak=True)
ham_time_evol = ham_evol(num_qubits, trotter_step=trotter_step, num_trotter_steps=num_trotter_steps, time=rescaled_total_time)

# num_trott_steps = int(np.ceil(8 * np.pi * np.abs(T) / rescaling_factor))
circ = QuantumCircuit(num_qubits, name="H")

for q in range(num_qubits):
    # phase_gate = Operator(pad_term(shift_1q(shift), num_qubits, q))
    phase_gate = Operator(shift_1q(shift*total_time))
    circ.append(phase_gate, [q])
    
circ.compose(ham_time_evol, range(num_qubits), inplace=True)

print(f'#Qubits {num_qubits}')
print(f'#Trotter steps {num_trotter_steps}')
print(f'#Gates {len([(op, _, _) for (op, _, _) in circ.data])}')
# print(circ)


#Qubits 3
#Trotter steps 100
#Gates 1003


### Comparing Circuit unitary operator to Exact

In [25]:
#* Circuit operators
trotterized_evolution_unitary = Operator(circ).data

#* Distance to exact unitary
dist_circ_unitary_vs_exact = np.linalg.norm(trotterized_evolution_unitary - exact_U_qt)

#* Distance to qutip trotter
dist_circ_vs_qutip_trotter = np.linalg.norm(trotterized_evolution_unitary - trott_U_qt)  # Correct without shift
circ_qutip_dag = qt.Qobj(trotterized_evolution_unitary) * qt.Qobj(trott_U_qt).dag() - qt.qeye(trotterized_evolution_unitary.shape[0])

print(f'Distance between circ unitary and exact {dist_circ_unitary_vs_exact}')
print(f'Distance between circ unitary and qutip trotter {dist_circ_vs_qutip_trotter}')
print(circ_qutip_dag)

Distance between circ unitary and exact 0.04150634298227804
Distance between circ unitary and qutip trotter 0.0
Quantum object: dims = [[8], [8]], shape = (8, 8), type = oper, isherm = True
Qobj data =
[[0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0.]]


### Fidelity of Circuit vs Exact

In [28]:
circ_with_ini_state = QuantumCircuit(num_qubits, name="H")
circ_with_ini_state.initialize(rand_ini_state, range(num_qubits))  #! Qiskit and Qutip order is not reversed for the state!
initial_state_from_circ = Statevector(circ_with_ini_state).data
energy_initial_state_from_circ_np = initial_state_from_circ.conj().T @ rescaled_hamiltonian_qt.data @ initial_state_from_circ
print('Energy of initial state:')
print(energy_initial_state_from_circ_np.real)

circ_with_ini_state.compose(ham_time_evol, range(num_qubits), inplace=True)
qiskit_end_state = Statevector(circ_with_ini_state).data  # Circ to state

print('Energy after time evolution (should not be changed):')
end_energy = qiskit_end_state.conj().T @ rescaled_hamiltonian_qt.data @ qiskit_end_state
print(end_energy.real)

print('QISKIT CIRCUIT FIDELITY')
qiskit_trott_evolved_state = Statevector(trott_evolved_state.full())
qiskit_exact_evolved_state = Statevector(exact_evolved_state.full())

print(f'To qutip exact {state_fidelity(qiskit_exact_evolved_state, qiskit_end_state)}')
print(f'To qutip Trotter {state_fidelity(qiskit_trott_evolved_state, qiskit_end_state)}')


Energy of initial state:
0.6875
Energy after time evolution (should not be changed):
0.6888257178930521
QISKIT CIRCUIT FIDELITY
To qutip exact 0.9998788797309984
To qutip Trotter 0.9999999999997906


### Trotterized Hamiltonian comparison, Qiskit vs Exact, Qutip vs Exact

In [ ]:
#* LOGM
# logU = logm(trotterized_evolution_unitary)  # log U = i 2pi H T / rescaling_factor  #! It's not unique if any eigenenergy < 0
# trotterized_ham_from_circuit = logU / (1j * T) #- shift * qt.qeye(hamiltonian_qt.shape[0])
# qiskit_trott_ham_spec = np.linalg.eigvalsh(trotterized_ham_from_circuit)
# spectrum_dist = np.linalg.norm(qiskit_trott_ham_spec - exact_spec)
# print(f'Spectrum distance {spectrum_dist}')

#### Qiskit gate angle convention! RXX(-2x) = exp(x)

In [ ]:
c = QuantumCircuit(2)
c.rxx(np.pi/2, 0, 1)
op = Operator(c)
print(op.data)
X = qt.sigmax()
XX = qt.tensor(X, X)
op_qt = expm(-1j*np.pi* XX / 4)
print(op_qt)


### Compare qiskit and qutip gates in the Trotter

In [ ]:
# np.random.seed(138)
num_qubits = 10
# dist_qutip_qiskit = 0.
# while dist_qutip_qiskit < 1:
step_size = np.random.rand()
step_size = 1.1
print(f'RXX angle , {step_size}')
i = np.random.randint(0, num_qubits)
j = (i + 1) % num_qubits
print(f'Sites {i, j}')

#* Qutip
XX = pad_term([qt.sigmax(), qt.sigmax()], num_qubits, i)
eXX = expm(1j*step_size*XX.full())

# XX_byhand_reversed = qt.tensor([qt.qeye(2), qt.sigmax(), qt.sigmax()])
# eXX_byhand_reversed = expm(1j*step_size*XX_byhand_reversed.full())
print('Qutip')
print(eXX)
#* Qiskit
rxx_circ = QuantumCircuit(num_qubits)
rxx_circ.rxx(-2 * step_size, i, j)
rxx_op = Operator(rxx_circ)
print('Qiskit')
print(rxx_op)

dist_qutip_qiskit = np.linalg.norm(eXX - rxx_op.data)
# dist_qutip_qiskit_reversed = np.linalg.norm(eXX_byhand_reversed - rxx_op.data)
print(f'Distance between Qutip and Qiskit RXX {dist_qutip_qiskit}')
# print(f'Distance between Qutip and Qiskit RXX reversed {dist_qutip_qiskit_reversed}')

